In [ ]:
# %pip install python-dotenv
# %pip install pandas
# %pip install requests
# %pip install pyshacl

In [1]:
import requests
import pandas as pd
from io import StringIO

In [4]:
building1_sparql_endpoint = 'http://localhost:7400/repositories/dsm-mpc'

In [ ]:
sparql_query = """
PREFIX ref: <https://brickschema.org/schema/Brick/ref#>
PREFIX brick: <https://brickschema.org/schema/Brick#>
PREFIX brick-ev: <https://w3id.org/b4b/schemes/ext/brick-ev#>
PREFIX sba: <https://w3id.org/b4b/schemes/algorithms/sba#> .
PREFIX mpc: <https://w3id.org/b4b/schemes/algorithms/mpc#> .
PREFIX forc: <https://w3id.org/b4b/schemes/algorithms/forc#> .


SELECT ?WindSpeedForecast ?SolarRadianceForecast 
?OutdoorTemperatureForecast ?BuildingLoad 
?OutdoorTemperature ?WindSpeed ?SolarRadiance 

WHERE { 
    ?windForc a  forc:WindSpeedForecast .
    ?windForc  ref:hasExternalReference ?windForcExt .
    ?windForcExt ref:hasForecastTimeseriesId ?WindSpeedForecast .

    ?ghiForc a  forc:SolarRadianceForecast .
    ?ghiForc  ref:hasExternalReference ?ghiForcExt .
    ?ghiForcExt ref:hasForecastTimeseriesId ?SolarRadianceForecast .

    ?tempForc a  forc:OutdoorTemperatureForecast .
    ?tempForc  ref:hasExternalReference ?tempForcExt .
    ?tempForcExt ref:hasForecastTimeseriesId ?OutdoorTemperatureForecast .

    ?bLoad a  brick:Building_Electrical_Meter .
    ?bLoad  ref:hasExternalReference ?bLoadExt .
    ?bLoadExt ref:hasHistoricalTimeseriesId ?BuildingLoad .

    ?temp a  brick:Outside_Air_Temperature_Sensor .
    ?temp  ref:hasExternalReference ?tempExt .
    ?tempExt ref:hasHistoricalTimeseriesId ?OutdoorTemperature .

    ?windSp a  brick:Wind_Speed_Sensor .
    ?windSp  ref:hasExternalReference ?windSpExt .
    ?windSpExt ref:hasHistoricalTimeseriesId ?WindSpeed .

    ?ghi a  brick:Solar_Radiance_Sensor .
    ?ghi  ref:hasExternalReference ?ghiExt .
    ?ghiExt ref:hasHistoricalTimeseriesId ?SolarRadiance .
}
"""

query_params = {
    'query': sparql_query,
    'infer': 'false' 
}

response = requests.get(building1_sparql_endpoint, params=query_params)

if response.status_code == 200:
    text = pd.read_csv(StringIO(response.text), sep=",")
    df = pd.DataFrame(text, columns=[
        "WindSpeedForecast", 
        "SolarRadianceForecast", 
        "OutdoorTemperatureForecast", 
        "BuildingLoad", 
        "OutdoorTemperature", 
        "WindSpeed", 
        "SolarRadiance"], index=None)
else:
    print(f"Error: {response.status_code} - {response.reason}")
df_dict = df.to_dict()
with open("forcasts.env", "w") as file:
    for key, value in df_dict.items():
        file.write(f"{key}={value[0]}\n")
df

,WindSpeedForecast,SolarRadianceForecast,OutdoorTemperatureForecast,BuildingLoad,OutdoorTemperature,WindSpeed,SolarRadiance
0,3ede3817d96d7bb87865eaa7ac657860.Value,3ede3817d96d7bb87865eaa7ac657860.Value,3ede3817d96d7bb87865eaa7ac657860.Value,f30ca6c6d2b3111dee9066951e270141.Value,a683b494a806af9a8da4d35ae99ba763.Value,a683b494a806af9a8da4d35ae99ba763.Value,a683b494a806af9a8da4d35ae99ba763.Value


In [ ]:
def charge_station_config(station_id):

    sparql_query = f"""
    PREFIX ref: <https://brickschema.org/schema/Brick/ref#>
    PREFIX brick: <https://brickschema.org/schema/Brick#>
    PREFIX brick-ev: <https://w3id.org/b4b/schemes/ext/brick-ev#>
    PREFIX sba: <https://w3id.org/b4b/schemes/algorithms/sba#> .
    PREFIX mpc: <https://w3id.org/b4b/schemes/algorithms/mpc#> .
    PREFIX forc: <https://w3id.org/b4b/schemes/algorithms/forc#> .

    SELECT DISTINCT 
    ?chrgStId{station_id} 
    ?timeArrId{station_id} 
    ?timeDeptId{station_id} 
    ?arrivalSOCId{station_id} 
    ?evBatId{station_id} 
    ?vehicle_full_status_id{station_id} 
    ?charging_status_id{station_id} 
    ?energy_id{station_id} 
    ?visual_number_id{station_id}

    where {{ 
        
    ?s a brick:Electric_Vehicle_Charging_Port .
    ?s eva:physical_reference "{station_id}"  .
    ?s brick:hasPoint 
    ?chrgSt{station_id}, 
    ?timeArr{station_id}, 
    ?timeDept{station_id}, 
    ?arrivalSOC{station_id}, 
    ?evBat{station_id}, 
    ?vFull{station_id}, 
    ?chrgStatus{station_id}, 
    ?energy{station_id},
    ?visualNum{station_id} .
        
    ?chrgSt{station_id} a brick-ev:EV_Charging_Station_ID .
    ?chrgSt{station_id}  ref:hasExternalReference ?chrgStExt{station_id} .
    ?chrgStExt{station_id} ref:hasRealtimeControlId ?chrgStId{station_id} .
            
    ?timeArr{station_id} a brick-ev:EV_Arrival_Time .
    ?timeArr{station_id}  ref:hasExternalReference ?timeArrExt{station_id} .
    ?timeArrExt{station_id} ref:hasRealtimeTimeseriesId ?timeArrId{station_id} .
        
    ?timeDept{station_id} a brick-ev:EV_Departure_Time .
    ?timeDept{station_id}  ref:hasExternalReference ?timeDeptExt{station_id} .
    ?timeDeptExt{station_id} ref:hasRealtimeTimeseriesId ?timeDeptId{station_id} .
        
    ?arrivalSOC{station_id} a brick-ev:EV_Arrival_SOC .
    ?arrivalSOC{station_id}  ref:hasExternalReference ?arrivalSOC{station_id}Ext .
    ?arrivalSOC{station_id}Ext ref:hasRealtimeTimeseriesId ?arrivalSOCId{station_id} .

    ?evBat{station_id} a brick-ev:EV_Battery_Size .
    ?evBat{station_id}  ref:hasExternalReference ?evBatExt{station_id} .
    ?evBatExt{station_id} ref:hasRealtimeTimeseriesId ?evBatId{station_id} .

    ?vFull{station_id} a brick-ev:Vehicle_Full_Status .
    ?vFull{station_id}  ref:hasExternalReference ?vFullExt{station_id} .
    ?vFullExt{station_id} ref:hasRealtimeTimeseriesId ?vehicle_full_status_id{station_id} .

    ?vFull{station_id} a brick-ev:Vehicle_Full_Status .
    ?vFull{station_id}  ref:hasExternalReference ?vFullExt{station_id} .
    ?vFullExt{station_id} ref:hasRealtimeTimeseriesId ?vehicle_full_status_id{station_id} .

    ?chrgStatus{station_id} a brick-ev:Charging_Session_Status .
    ?chrgStatus{station_id}  ref:hasExternalReference ?chrgStatusExt{station_id} .
    ?chrgStatusExt{station_id} ref:hasRealtimeTimeseriesId ?charging_status_id{station_id} .
            
    ?energy{station_id} a brick-ev:EV_Electric_Energy_Meter .
    ?energy{station_id}  ref:hasExternalReference ?energyExt{station_id} .
    ?energyExt{station_id} ref:hasHistoricalTimeseriesId ?energy_id{station_id} .

    ?visualNum{station_id} a brick-ev:EV_Charging_Session_Visual_Number .
    ?visualNum{station_id}  ref:hasExternalReference ?visualNumExt{station_id} .
    ?visualNumExt{station_id} ref:hasRealtimeTimeseriesId ?visual_number_id{station_id} .

    }}
    """

    query_params = {
        'query': sparql_query,
        'infer': 'false' 
    }

    response = requests.get(building1_sparql_endpoint, params=query_params)

    if response.status_code == 200:
        text = pd.read_csv(StringIO(response.text), sep=",")
        df = pd.DataFrame(text, columns=
                          [f"chrgStId{station_id}", 
                           f"timeArrId{station_id}", 
                           f"timeDeptId{station_id}", 
                           f"arrivalSOCId{station_id}", 
                           f"evBatId{station_id}", 
                           f"vehicle_full_status_id{station_id}", 
                           f"charging_status_id{station_id}", 
                           f"energy_id{station_id}", 
                           f"visual_number_id{station_id}"], index=None)
    else:
        print(f"Error: {response.status_code} - {response.reason}")
    df_dict = df.to_dict()
    with open(f"feedback_id{station_id}.env", "w") as file:
        for key, value in df_dict.items():
            file.write(f"{key}={value[0]}\n")
    return df

In [7]:
df = charge_station_config(1)
df

,chrgStId1,timeArrId1,timeDeptId1,arrivalSOCId1,evBatId1,vehicle_full_status_id1,charging_status_id1,energy_id1,visual_number_id1
0,1,welkom_kropman_records,welkom_kropman_records,welkom_kropman_records,welkom_kropman_records,759ca9cc4f3bf8f552084d67f25d4589.vehicle_full_...,759ca9cc4f3bf8f552084d67f25d4589.status,759ca9cc4f3bf8f552084d67f25d45899.kwh,759ca9cc4f3bf8f552084d67f25d4589.visual_number


In [8]:
df = charge_station_config(2)
df

,chrgStId2,timeArrId2,timeDeptId2,arrivalSOCId2,evBatId2,vehicle_full_status_id2,charging_status_id2,energy_id2,visual_number_id2
0,2,welkom_kropman_records,welkom_kropman_records,welkom_kropman_records,welkom_kropman_records,1cb94c6d3ecd7c90b08317ad11121a31.vehicle_full_...,1cb94c6d3ecd7c90b08317ad11121a31.status,1cb94c6d3ecd7c90b08317ad11121a31c.kwh,1cb94c6d3ecd7c90b08317ad11121a31.visual_number


In [9]:
df = charge_station_config(3)
df

,chrgStId3,timeArrId3,timeDeptId3,arrivalSOCId3,evBatId3,vehicle_full_status_id3,charging_status_id3,energy_id3,visual_number_id3
0,3,welkom_kropman_records,welkom_kropman_records,welkom_kropman_records,welkom_kropman_records,f10dc57115f3546ff2ccb745362c73c9.vehicle_full_...,f10dc57115f3546ff2ccb745362c73c9.status,f10dc57115f3546ff2ccb745362c73c96.kwh,f10dc57115f3546ff2ccb745362c73c9.visual_number


In [10]:
df = charge_station_config(4)
df

,chrgStId4,timeArrId4,timeDeptId4,arrivalSOCId4,evBatId4,vehicle_full_status_id4,charging_status_id4,energy_id4,visual_number_id4
0,4,welkom_kropman_records,welkom_kropman_records,welkom_kropman_records,welkom_kropman_records,83a9cd04a4852e57bc3918762d1a049f.vehicle_full_...,83a9cd04a4852e57bc3918762d1a049f.status,83a9cd04a4852e57bc3918762d1a049f3.kwh,83a9cd04a4852e57bc3918762d1a049f.visual_number


In [11]:
input_files = ["feedback_id1.env", "feedback_id2.env", "feedback_id3.env", "feedback_id4.env"]

output_file = "feedback.env"

with open(output_file, "w") as outfile:
    for file in input_files:
        with open(file, "r") as infile:
            outfile.write(infile.read()) 
            outfile.write("\n")

print(f"Merged files into {output_file}")


Merged files into feedback.env
